In [1]:
import baseline

In [2]:
X, y = baseline.baselineXy()

In [3]:
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer, Normalizer, PowerTransformer, RobustScaler, SplineTransformer, StandardScaler, MinMaxScaler
td = {
 'sdoif_mean_id': QuantileTransformer,
 'sdoif_mean_sq': RobustScaler,
 'elevation_mean_sqrt': QuantileTransformer,
 'procurv_mean_id': StandardScaler,
 'placurv_mean_id': Normalizer,
 'lsfactor_mean_id': Normalizer,
 'slope_mean_id': RobustScaler,
 'twi_mean_id': Normalizer,
 'twi_mean_sqrt': QuantileTransformer,
 'aspect_mean_id': QuantileTransformer
}

In [4]:
for key, t in td.items():
    X[key] = t().fit_transform(X[[key]])
X = X.to_numpy()

In [5]:
groups = [
    [1, 2, 6, 7],
    [3, 8],
    [4, 5, 9, 10],
    [11, 12],
    [13],
    [14, 15],
    [16, 17, 21, 22],
    [18, 23],
    [19, 20, 24, 25],
    list(range(1, 26))
]

def get_group_mean(df, col, group):
    cols = [f"{i}_{col}" for i in group]
    return df[cols].to_numpy().mean(axis=1)

In [6]:
import baseline
X, y = baseline.originalXy()
get_group_mean(X, "slope", groups[0])

array([37.73571  , 31.7261425, 49.1132025, ..., 39.9281625, 35.24104  ,
       20.02481  ])

In [28]:
import pandas as pd
import numpy as np

def preprocess(X):
    grid_groups = pd.DataFrame()

    def normalize(x):
        return (x - x.min()) / (x.max() - x.min())

    def special_normalize(x):
        return ((x - x.min()) / (x.max() - x.min()) + 0.01) / 1.01

    fns = [(np.array, 'id'), (np.sqrt, 'sqrt'), (np.square, 'sq'), (np.log, 'log')]
    print(len(baseline.continuous) * len(groups) * len(fns))
    for col in baseline.continuous:
        for i, group in enumerate(groups):
            for fn, name in fns:
                this = f"{col}_g{i}_{name}"
                if name == 'log':
                    grid_groups[this] = fn(special_normalize(get_group_mean(X, col, group)))
                else:
                    grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))

    import dfcols
    def most_freq(df, col):
        matrix = df[dfcols.all_square_cols(col)].to_numpy()
        return np.array(list(map(np.argmax, map(np.bincount, matrix))))

    cat = "geology"
    # get most freq category
    X[cat] = most_freq(X, cat)
    grid_groups[cat] = X[cat].map({
        1: "Weathered Cretaceous granitic rocks",
        2: "Weathered Jurassic granite rocks",
        3: "Weathered Jurassic tuff and lava",
        4: "Weathered Cretaceous tuff and lava",
        5: "Quaternary deposits",
        6: "Fill",
        7: "Weathered Jurassic sandstone, siltstone and mudstone"
    })
    return grid_groups
grid_groups = preprocess(X)

320


/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [29]:
grid_groups

,sdoif_g0_id,sdoif_g0_sqrt,sdoif_g0_sq,sdoif_g0_log,sdoif_g1_id,sdoif_g1_sqrt,sdoif_g1_sq,sdoif_g1_log,sdoif_g2_id,sdoif_g2_sqrt,...,aspect_g7_log,aspect_g8_id,aspect_g8_sqrt,aspect_g8_sq,aspect_g8_log,aspect_g9_id,aspect_g9_sqrt,aspect_g9_sq,aspect_g9_log,geology
0,0.680410,0.824870,0.462958,-0.380419,0.680235,0.824763,0.462719,-0.380674,0.680087,0.824674,...,-1.118586,0.323977,0.569190,0.104961,-1.106634,0.303978,0.551342,0.092402,-1.168384,Weathered Jurassic tuff and lava
1,0.960227,0.979912,0.922036,-0.040175,0.960051,0.979822,0.921698,-0.040357,0.959901,0.979745,...,-0.675332,0.481153,0.693652,0.231508,-0.720950,0.500208,0.707254,0.250208,-0.682887,Weathered Jurassic tuff and lava
2,0.979967,0.989933,0.960336,-0.020034,0.980065,0.989982,0.960528,-0.019935,0.980189,0.990045,...,-0.326154,0.722408,0.849946,0.521873,-0.321368,0.790661,0.889191,0.625145,-0.232268,Weathered Jurassic granite rocks
3,0.029984,0.173158,0.000899,-3.229237,0.030168,0.173691,0.000910,-3.224625,0.030344,0.174196,...,-0.846976,0.410311,0.640555,0.168355,-0.876712,0.466744,0.683187,0.217850,-0.750726,Weathered Jurassic granite rocks
4,0.690175,0.830768,0.476342,-0.366375,0.689855,0.830575,0.475900,-0.366832,0.689496,0.830359,...,-1.019687,0.329545,0.574060,0.108600,-1.090100,0.329019,0.573602,0.108253,-1.091650,Quaternary deposits
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10859,0.646353,0.803961,0.417773,-0.431006,0.646819,0.804251,0.418375,-0.430297,0.647233,0.804508,...,-1.088610,0.294239,0.542438,0.086576,-1.199894,0.308574,0.555495,0.095218,-1.153850,Weathered Jurassic granite rocks
10860,0.857721,0.926132,0.735685,-0.151835,0.857601,0.926067,0.735479,-0.151974,0.857472,0.925998,...,-0.905288,0.481153,0.693652,0.231508,-0.720950,0.401679,0.633782,0.161346,-0.897461,Weathered Jurassic tuff and lava
10861,0.739961,0.860210,0.547543,-0.297684,0.739836,0.860137,0.547358,-0.297851,0.739731,0.860076,...,-0.920145,0.364499,0.603738,0.132860,-0.992116,0.360548,0.600456,0.129995,-1.002723,Weathered Jurassic tuff and lava
10862,0.944510,0.971859,0.892099,-0.056508,0.944649,0.971930,0.892361,-0.056362,0.944817,0.972017,...,-0.320025,0.709069,0.842063,0.502780,-0.339748,0.698702,0.835884,0.488185,-0.354270,Weathered Jurassic granite rocks


In [9]:
from finn.feat_trans import test_feats
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier
from catboost import CatBoostClassifier
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import f1_score
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.linear_model import LogisticRegression

# estimators = [
#     ('rf', RandomForestClassifier(n_estimators=10, random_state=42)),
#     ('svr', make_pipeline(StandardScaler(),
#                           LinearSVC(random_state=42)))
# ]
models = [LogisticRegression, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier, CatBoostClassifier]

"""
k = 3
kf = KFold(n_splits=k)
for m in models:
    f1m = 0
    for train_ind, test_ind in kf.split(grid_groups.to_numpy()):
        X_train, X_test = grid_groups.to_numpy()[train_ind], grid_groups.to_numpy()[test_ind]
        y_train, y_test = y[train_ind], y[test_ind]
        X_train, y_train = SMOTE().fit_resample(X_train, y_train)
        mym = m().fit(X_train, y_train)
        probs = mym.predict_proba(X_test)[:, 1]
        y_pred = np.array(probs > 0.4, dtype=int)
        #print(y_pred)
        f1 = f1_score(y_test, y_pred)
        print(f1)
        f1m += f1 / k
    print(m.__name__, f1m)
"""

'\nk = 3\nkf = KFold(n_splits=k)\nfor m in models:\n    f1m = 0\n    for train_ind, test_ind in kf.split(grid_groups.to_numpy()):\n        X_train, X_test = grid_groups.to_numpy()[train_ind], grid_groups.to_numpy()[test_ind]\n        y_train, y_test = y[train_ind], y[test_ind]\n        X_train, y_train = SMOTE().fit_resample(X_train, y_train)\n        mym = m().fit(X_train, y_train)\n        probs = mym.predict_proba(X_test)[:, 1]\n        y_pred = np.array(probs > 0.4, dtype=int)\n        #print(y_pred)\n        f1 = f1_score(y_test, y_pred)\n        print(f1)\n        f1m += f1 / k\n    print(m.__name__, f1m)\n'

In [11]:
from sklearn.feature_selection import SelectFromModel
def tree_features(X, n):
    clf = ExtraTreesClassifier(n_estimators=50)
    clf = clf.fit(X, y)
    model = SelectFromModel(clf, prefit=True, max_features=n)

    X_clf = model.transform(X)
    return test_feats(ExtraTreesClassifier(n_estimators=50), X_clf, y)

In [13]:
def something():
    for sc in [StandardScaler, MinMaxScaler, QuantileTransformer, PowerTransformer]:
        hX = sc().fit_transform(gg)
        clf = ExtraTreesClassifier()
        clf = clf.fit(hX, y)
        model = SelectFromModel(clf, prefit=True)
        X_clf = model.transform(hX)
        print(sc.__name__, test_feats(ExtraTreesClassifier(n_estimators=50), X_clf, y))

In [14]:
import lightgbm as lgb

In [15]:
def lgb_test(x, y):
    kf = KFold(n_splits=3)
    f1m = 0
    for train_ind, test_ind in kf.split(x):
        X_train, X_test = x[train_ind], x[test_ind]
        y_train, y_test = y[train_ind], y[test_ind]
        X_train, y_train = SMOTE().fit_resample(X_train, y_train)
        X_train, X_test = StandardScaler().fit_transform(X_train), StandardScaler().fit_transform(X_test)
        ds = lgb.Dataset(X_train, label=y_train)
        bst = lgb.train({'num_leaves': 30, 'objective': 'binary', 'metric': 'auc', 'n_jobs': 2}, ds)
        y_pred = bst.predict(X_test)
        y_pred = np.array(y_pred > 0.5, dtype=int)
        f1 = f1_score(y_test, y_pred)
        f1m += f1 / 3
    return f1m

In [17]:
lgb_test(grid_groups.drop(["geology"], axis=1, inplace=False).to_numpy(), y)

[LightGBM] [Info] Number of positive: 5456, number of negative: 5456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.037240 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 81600
[LightGBM] [Info] Number of data points in the train set: 10912, number of used features: 320
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5418, number of negative: 5418
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.045467 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 81600
[LightGBM] [Info] Number of data points in the train set: 10836, number of used features: 320
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5422, number of negative: 5422
[LightGBM] [Warning] Auto-choosing col-wise mul

0.6025762321404486

In [18]:
import featuretools as ft

In [19]:
es = ft.EntitySet(id = 'gridgroups')
grid_groups['id'] = grid_groups.index
es = ft.EntitySet(id = 'gridgroups')
es = es.add_dataframe(dataframe_name='gg', dataframe =grid_groups, index='id')
es.normalize_dataframe(base_dataframe_name='gg', new_dataframe_name='2nd', index='geology')

Entityset: gridgroups
  DataFrames:
    gg [Rows: 10864, Columns: 322]
    2nd [Rows: 7, Columns: 1]
  Relationships:
    gg.geology -> 2nd.geology

In [20]:
feature_matrix, feature_names = ft.dfs(
    entityset=es, 
    target_dataframe_name = 'gg',
    max_depth=2
)

len(feature_names)

2242

In [21]:
feats = StandardScaler().fit_transform(feature_matrix.drop(["geology"], axis=1, inplace=False))
feats

array([[-0.37303177, -0.20639589, -0.60936445, ..., -0.86115675,
         0.83649743,  0.85975046],
       [ 1.12698306,  0.88288997,  1.47308477, ..., -0.86115675,
         0.83649743,  0.85975046],
       [ 1.23280512,  0.95329685,  1.64681993, ...,  0.90482352,
        -1.12893699, -0.98071659],
       ...,
       [-0.05379693,  0.04189499, -0.2256762 , ..., -0.86115675,
         0.83649743,  0.85975046],
       [ 1.04272735,  0.82631275,  1.3372845 , ...,  0.90482352,
        -1.12893699, -0.98071659],
       [ 0.56833535,  0.49857612,  0.61450997, ..., -0.86115675,
         0.83649743,  0.85975046]])

In [23]:
lgb_test(feats, y)

[LightGBM] [Info] Number of positive: 5456, number of negative: 5456
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.185232 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 96644
[LightGBM] [Info] Number of data points in the train set: 10912, number of used features: 2233
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5418, number of negative: 5418
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.471047 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 96644
[LightGBM] [Info] Number of data points in the train set: 10836, number of used features: 2233
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5422, number

0.6157702746422324

In [24]:
from boruta import BorutaPy
feat_selector = BorutaPy(
    verbose=2,
    estimator=ExtraTreesClassifier(),
    n_estimators='auto',
    max_iter=10  # number of iterations to perform
)
feat_selector.fit(feats, y)

Iteration: 	1 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	2 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	3 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	4 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	5 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	6 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	7 / 10
Confirmed: 	0
Tentative: 	2241
Rejected: 	0
Iteration: 	8 / 10
Confirmed: 	248
Tentative: 	284
Rejected: 	1709
Iteration: 	9 / 10
Confirmed: 	248
Tentative: 	284
Rejected: 	1709


BorutaPy finished running.

Iteration: 	10 / 10
Confirmed: 	248
Tentative: 	71
Rejected: 	1709


BorutaPy(estimator=ExtraTreesClassifier(n_estimators=326,
                                        random_state=RandomState(MT19937) at 0x2B64F3EEDD40),
         max_iter=10, n_estimators='auto',
         random_state=RandomState(MT19937) at 0x2B64F3EEDD40, verbose=2)

In [25]:
feat_selector.transform(feats).shape

(10864, 248)

In [26]:
lgb_test(feat_selector.transform(feats), y)

[LightGBM] [Info] Number of positive: 5456, number of negative: 5456
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.050928 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63237
[LightGBM] [Info] Number of data points in the train set: 10912, number of used features: 248
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5418, number of negative: 5418
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.019002 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 63238
[LightGBM] [Info] Number of data points in the train set: 10836, number of used features: 248
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=0.000000
[LightGBM] [Info] Number of positive: 5422, number of negative: 5422
[LightGBM] [Warning] Auto-choosing col-wise mul

0.5136524826262826

In [27]:
cb = CatBoostClassifier(iterations=400, learning_rate=0.2, depth=6)
test_feats(cb, feat_selector.transform(feats), y)

0:	learn: 0.5833996	total: 181ms	remaining: 1m 12s
1:	learn: 0.5113672	total: 226ms	remaining: 45s
2:	learn: 0.4737487	total: 271ms	remaining: 35.9s
3:	learn: 0.4495428	total: 322ms	remaining: 31.9s
4:	learn: 0.4260257	total: 372ms	remaining: 29.4s
5:	learn: 0.4089173	total: 428ms	remaining: 28.1s
6:	learn: 0.4006509	total: 474ms	remaining: 26.6s
7:	learn: 0.3889901	total: 526ms	remaining: 25.8s
8:	learn: 0.3808598	total: 600ms	remaining: 26.1s
9:	learn: 0.3737514	total: 658ms	remaining: 25.7s
10:	learn: 0.3680432	total: 702ms	remaining: 24.8s
11:	learn: 0.3636558	total: 746ms	remaining: 24.1s
12:	learn: 0.3585111	total: 815ms	remaining: 24.3s
13:	learn: 0.3539373	total: 864ms	remaining: 23.8s
14:	learn: 0.3505214	total: 919ms	remaining: 23.6s
15:	learn: 0.3474621	total: 982ms	remaining: 23.6s
16:	learn: 0.3441163	total: 1.05s	remaining: 23.8s
17:	learn: 0.3407552	total: 1.1s	remaining: 23.4s
18:	learn: 0.3376195	total: 1.15s	remaining: 23s
19:	learn: 0.3341574	total: 1.21s	remaining: 

0.6794704428866957

In [34]:
X_test = baseline.origtestX()
test_gg = preprocess(X_test)
es = ft.EntitySet(id = 'gridgroups')
test_gg['id'] = test_gg.index
es = ft.EntitySet(id = 'gridgroups')
es = es.add_dataframe(dataframe_name='gg', dataframe =test_gg, index='id')
es.normalize_dataframe(base_dataframe_name='gg', new_dataframe_name='2nd', index='geology')
feature_matrix, feature_names = ft.dfs(
    entityset=es, 
    target_dataframe_name = 'gg',
    max_depth=2
)
feats = StandardScaler().fit_transform(feature_matrix.drop(["geology"], axis=1, inplace=False))
short_feats = feat_selector.transform(feats)
y_pred = cb.predict(short_feats)

320


/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grid_groups[this] = fn(normalize(get_group_mean(X, col, group)))
/tmp/ipykernel_21397/3501114163.py:22: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.co

In [35]:
y_pred

array([1, 0, 0, ..., 0, 0, 1])

In [36]:
test = pd.read_csv("data/Test.csv")
sub_file = pd.DataFrame({'Sample_ID': test.Sample_ID, 'Label': y_pred})
sub_file.to_csv('finn/cat_fs.csv', index = False)
sub_file.head()

,Sample_ID,Label
0,10865,1
1,10866,0
2,10867,0
3,10868,0
4,10869,1


In [ ]:
from sklearn.ensemble import StackingClassifier
hailmary = StackingClassifier([cv=
    
])